# Statistical Analysis

Statistics is the science of collecting, analyzing, presenting, and interpreting data.
## Descriptive Analysis

In [18]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Doctorate Recipients by Demography--- Doctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df = pd.read_csv('../../Users/gitData/SED_export_table_2020-11-12T19_44_35.595Z.csv', skiprows=[0], header=[1])

# Removing 'Total values' --- for calculating statistical purposes
df1 = df['Detailed Fields']!='Total for selected values'
df = df[df1]
df2 = df['Race and Ethnicity']!='Total for selected values'
df = df[df2]
df3 = df['Citizenship']!='Total for selected values'
df = df[df3]
df4 = df['Sex']!='Total for selected values'
df = df[df4]
df.head()

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,2018,2017,2016,2015,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
5,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,19,19,19,24,...,17,20,12,13,11,10,10,8,17,2
6,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Female,30,28,27,28,...,4,2,5,6,6,-,2,3,1,5
7,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Unknown sex,1,-,-,-,...,-,-,-,-,-,-,-,-,-,-
10,Science and engineering,Life sciences,Agricultural sciences and natural resources,"American Indian or Alaska Native, Not Hispanic...",U.S. citizen or permanent resident,Male,1,1,2,-,...,1,4,2,1,4,1,1,2,2,1
11,Science and engineering,Life sciences,Agricultural sciences and natural resources,"American Indian or Alaska Native, Not Hispanic...",U.S. citizen or permanent resident,Female,-,1,3,1,...,1,3,-,-,-,-,-,1,-,-


In [20]:
# checking the data types of the columns in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666 entries, 5 to 1305
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   S&E Fields          663 non-null    object
 1   Broad Fields        660 non-null    object
 2   Detailed Fields     660 non-null    object
 3   Race and Ethnicity  660 non-null    object
 4   Citizenship         660 non-null    object
 5   Sex                 660 non-null    object
 6   2018                660 non-null    object
 7   2017                660 non-null    object
 8   2016                660 non-null    object
 9   2015                660 non-null    object
 10  2014                660 non-null    object
 11  2013                660 non-null    object
 12  2012                660 non-null    object
 13  2011                660 non-null    object
 14  2010                660 non-null    object
 15  2009                660 non-null    object
 16  2008                660 n

In [21]:
# checking for duplicate values in the data
df.duplicated().sum()

2

In [22]:
# let's check for missing values in the data
round(df.isnull().sum() / df.isnull().count() * 100, 2)

S&E Fields            0.45
Broad Fields          0.90
Detailed Fields       0.90
Race and Ethnicity    0.90
Citizenship           0.90
Sex                   0.90
2018                  0.90
2017                  0.90
2016                  0.90
2015                  0.90
2014                  0.90
2013                  0.90
2012                  0.90
2011                  0.90
2010                  0.90
2009                  0.90
2008                  0.90
2007                  0.90
2006                  0.90
2005                  0.90
2004                  0.90
2003                  0.90
2002                  0.90
2001                  0.90
2000                  0.90
1999                  0.90
1998                  0.90
1997                  0.90
1996                  0.90
1995                  0.90
1994                  0.90
1993                  0.90
1992                  0.90
1991                  0.90
1990                  0.90
1989                  0.90
1988                  0.90
1

In [23]:
df = df.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Race and Ethnicity',
       'Citizenship', 'Sex'])
df.head()

2018  \
S&E Fields              Broad Fields  Detailed Fields                             Race and Ethnicity                                 Citizenship                        Sex                
Science and engineering Life sciences Agricultural sciences and natural resources Hispanic or Latino                                 U.S. citizen or permanent resident Male          19   
                                                                                                                                                                        Female        30   
                                                                                                                                                                        Unknown sex    1   
                                                                                  American Indian or Alaska Native, Not Hispanic ... U.S. citizen or permanent resident Male           1   
                                                                                                                                                                        Female         -   

                                                                                                                                                                                    2017  \
S&E Fields              Broad Fields  Detailed Fields                             Race and Ethnicity                                 Citizenship                        Sex                
Science and engineering Life sciences Agricultural sciences and natural resources Hispanic or Latino                                 U.S. citizen or permanent resident Male          19   
                                                                                                                                                                        Female        28   
                                                                                                                                                                        Unknown sex    -   
                                                                                  American Indian or Alaska Native, Not Hispanic ... U.S. citizen or permanent resident Male           1   
                                                                                                                                                                        Female         1   

                                                                                                                                                                                    2016  \
S&E Fields              Broad Fields  Detailed Fields                             Race and Ethnicity                                 Citizenship                        Sex                
Science and engineering Life sciences Agricultural sciences and natural resources Hispanic or Latino                                 U.S. citizen or permanent resident Male          19   
                                                                                                                                                                        Female        27   
                                                                                                                                                                        Unknown sex    -   
                                                                                  American Indian or Alaska Native, Not Hispanic ... U.S. citizen or permanent resident Male           2   
                                                                                                                                                                        Female         3   

                                                                                                                                                                                    2015  \
S&E Fields              Broad Fields  Det

In [4]:
df = df.stack().to_frame()
df = df.reset_index()
df.head()

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,level_6,0
0,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2018,19
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2017,19
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2016,19
3,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2015,24
4,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2014,24


In [5]:
df.columns = ['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Race and Ethnicity',
       'Citizenship', 'Sex', 'Year', 'Number']
df.head()
#df.columns
df.shape

(25740, 8)

In [6]:
df3 = df['Number']!='-'
df = df[df3]
df.dtypes

S&E Fields            object
Broad Fields          object
Detailed Fields       object
Race and Ethnicity    object
Citizenship           object
Sex                   object
Year                  object
Number                object
dtype: object

In [7]:
df['Number']=df['Number'].str.replace(',','')  # Removing ',' from numbers to change object to int

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19577 entries, 0 to 25728
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   S&E Fields          19577 non-null  object
 1   Broad Fields        19577 non-null  object
 2   Detailed Fields     19577 non-null  object
 3   Race and Ethnicity  19577 non-null  object
 4   Citizenship         19577 non-null  object
 5   Sex                 19577 non-null  object
 6   Year                19577 non-null  object
 7   Number              19577 non-null  object
dtypes: object(8)
memory usage: 1.3+ MB


In [8]:
df["Number"] = df["Number"].astype(str).astype(int)
df.dtypes
#df["Number"] = pd.to_numeric(df["Number"])

S&E Fields            object
Broad Fields          object
Detailed Fields       object
Race and Ethnicity    object
Citizenship           object
Sex                   object
Year                  object
Number                 int32
dtype: object

### Common data manipulation tasks

1. Selecting/Indexing
2. Filtering
3. Sorting
4. Mutating/Conditionally adding documents
5. Groupby/Summarize

In [9]:
# selecting/Indexing 
df1 =  df.iloc[1:10, :]

df1.head(15)

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,Year,Number
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2017,19
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2016,19
3,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2015,24
4,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2014,24
5,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2013,25
6,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2012,24
7,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2011,26
8,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2010,11
9,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2009,14


In [10]:
# Filtering
df2 = df[df["Broad Fields"] == "Life sciences"]

df2.head()

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,Year,Number
0,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2018,19
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2017,19
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2016,19
3,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2015,24
4,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2014,24


In [11]:
# Sorting
df1.sort_values(["Number"],  ascending=False)

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,Year,Number
7,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2011,26
5,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2013,25
3,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2015,24
4,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2014,24
6,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2012,24
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2017,19
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2016,19
9,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2009,14
8,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2010,11


In [12]:
df[['Year', 'Number']].groupby(["Year"]).sum()

,Number
Year,
1980,31019
1981,31355
1982,31108
1983,31280
1984,31334
1985,31295
1986,31897
1987,32365
1988,33497


In [13]:
df[['S&E Fields',"Year", 'Number']].groupby(['S&E Fields',"Year"]).mean()

Number
S&E Fields                  Year            
Non-science and engineering 1980   79.813665
                            1981   83.721854
                            1982   79.819355
                            1983   77.471338
                            1984   70.461988
...                                      ...
Science and engineering     2014  111.936639
                            2015  114.701950
                            2016  112.661202
                            2017  113.445055
                            2018  117.297222

[78 rows x 1 columns]

In [14]:
df[['S&E Fields',"Year", 'Number']].groupby(['S&E Fields',"Year"]).sum()

Number
S&E Fields                  Year        
Non-science and engineering 1980   12850
                            1981   12642
                            1982   12372
                            1983   12163
                            1984   12049
...                                  ...
Science and engineering     2014   40633
                            2015   41178
                            2016   41234
                            2017   41294
                            2018   42227

[78 rows x 1 columns]

In [15]:
df.head()

,S&E Fields,Broad Fields,Detailed Fields,Race and Ethnicity,Citizenship,Sex,Year,Number
0,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2018,19
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2017,19
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2016,19
3,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2015,24
4,Science and engineering,Life sciences,Agricultural sciences and natural resources,Hispanic or Latino,U.S. citizen or permanent resident,Male,2014,24


In [16]:
df.groupby('Broad Fields').sum()

,Number
Broad Fields,
Education,243826
Engineering,234909
Humanities and arts,183230
Life sciences,340831
Mathematics and computer sciences,87629
Other non-S&E,89478
Physical sciences and earth sciences,175818
Psychology and social sciences,282035


In [17]:
df.groupby('Broad Fields')['Number'].sum()

Broad Fields
Education                               243826
Engineering                             234909
Humanities and arts                     183230
Life sciences                           340831
Mathematics and computer sciences        87629
Other non-S&E                            89478
Physical sciences and earth sciences    175818
Psychology and social sciences          282035
Name: Number, dtype: int32